In [1]:
!pip install earthengine-api
import ee

# Authenticate and initialize Earth Engine
try:
  ee.Initialize() # Attempts to initialize with existing credentials
except ee.EEException:
  ee.Authenticate() # Authenticates if no valid credentials found
  ee.Initialize(project='ee-mmdbzi1996') #

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.0/455.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 89.5 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.8.0
    Uninstalling google-api-python-client-1.8.0:
      Successfully uninstalled google-api-python-client-1.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.149.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json
with open('/content/drive/MyDrive/Articles/SoilSalinity/points.geojson') as f:
  data = json.load(f)
points=[x['geometry']['coordinates'] for x in data['features']]

In [4]:
AOI = ee.Geometry.MultiPoint(points,'EPSG:4326')
START_DATE = '2020-09-20'
END_DATE = '2020-10-26'
CLOUD_FILTER = 50
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [5]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [6]:
s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)
len(s2_sr_cld_col_eval.getInfo()['features'])

/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


32

In [7]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))


In [8]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [9]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))
    mask=is_cld_shdw.eq(0)
    mod_img=img.select('B.*').multiply(1e-4).updateMask(mask)

    # mod_img.set({'date':img.getInfo()['properties']['system:index'][:8]})
    # Add the final cloud-shadow mask to the image.
    return mod_img.copyProperties(img)

In [15]:
def add_bands(images):
  # Normalized Difference Vegetation Index
  NDVI=images.normalizedDifference(['B8','B4']).rename('NDVI');
  # Normalized Difference Salinity Index
  NDSI=images.normalizedDifference(['B4','B8']).rename('NDSI');
  # Vegetation Soil Salinity Index
  VSSI=images.expression(
    '(2*G)-5*(R+NIR)',{
      'G':images.select('B3'),
      'R':images.select('B4'),
      'NIR':images.select('B8')}).rename('VSSI');
  albedo= images.expression(
    '(0.356*B)+(0.130*R)+(0.373*NIR)+(0.085*SWIR1)',{
    'B':images.select('B2'),
    'R':images.select('B4'),
    'NIR':images.select('B8'),
    'SWIR1':images.select('B11')}).rename('albedo');
#  Canopy Response Salinity Index CRSI
  CRSI = images.expression(
    'sqrt(((NIR*G)-(G*R))/((NIR*G)+(G*R)))',{
    'G':images.select('B3'),
    'R':images.select('B4'),
    'NIR':images.select('B8')}).rename('CRSI');
  # // Brightness Index
  BI = images.expression(
    'sqrt((R**2)+(NIR**2))',{
    'R':images.select('B4'),
    'NIR':images.select('B8')}).rename('BI');
  #  Salinity Index SI
  SI=images.expression(
    'R*G/B',{
    'R':images.select('B4'),
    'G':images.select('B3'),
    'B':images.select('B2')}).rename('SI');
  # Salinity Index 1
  SI1=images.expression(
    'sqrt(G*R)',{
    'G':images.select('B3'),
    'R':images.select('B4')}).rename('SI1');
# Salinity Index 2
  SI2=images.expression(
    'sqrt(NIR*R)',{
    'NIR':images.select('B8'),
    'R':images.select('B4')}).rename('SI2');
  # Salinity Index 3
  SI3=images.expression(
    'sqrt((G**2)+(NIR**2)+(R**2))',{
    'G':images.select('B3'),
    'R':images.select('B4'),
    'NIR':images.select('B8')}).rename('SI3');
  # Salinity Index 4
  SI4=images.expression(
    'sqrt((G**2)+(R**2))',{
    'G':images.select('B3'),
    'R':images.select('B4')}).rename('SI4');
  #  Salinity Index 5
  SI5=images.expression(
    'B/R',{
    'B':images.select('B2'),
    'R':images.select('B4')}).rename('SI5');
  # Ratio Spectral Index
  RSI=images.expression(
    'R/NIR',{
    'NIR':images.select('B8'),
    'R':images.select('B4')}).rename('RSI');
  # Differential Vegetation Index
  DVI=images.expression(
    'NIR-R',{
    'NIR':images.select('B8'),
    'R':images.select('B4')}).rename('DVI');
  # Intensity Index 1
  II1=images.expression(
    '(G+R)/2',{
    'G':images.select('B3'),
    'R':images.select('B4')}).rename('II1');
  # Intensity Index 2
  II2=images.expression(
    '(G+R+NIR)/2',{
    'G':images.select('B3'),
    'NIR':images.select('B8'),
    'R':images.select('B4')}).rename('II2');
  # Simple Ratio
  SR=images.expression(
    '(R-NIR)/(G+NIR)',{
    'G':images.select('B3'),
    'NIR':images.select('B8'),
    'R':images.select('B4')}).rename('SR');
    # // Enhanced vegetation Index
  EVI=images.expression(
      '2.5*(NIR-R)/(NIR+1+(6*R)-(7.5*B))',{
      'B':images.select('B2'),
      'NIR':images.select('B8'),
      'R':images.select('B4')}).rename('EVI');
    # // Rational Vegetation Index
  RVI=images.expression(
      'NIR/R',{
        'NIR':images.select('B8'),
        'R':images.select('B4')}).rename('RVI')
  #  Soil Adjusted Vegetation Index
  SAVI=images.expression(
      '1.5*(NIR-R)/(NIR+R+0.5)',{
        'NIR':images.select('B8'),
        'R':images.select('B4')}).rename('SAVI')
  return images.addBands([NDVI,NDSI,albedo,CRSI,VSSI,BI,SI,SI1,SI2,SI3,SI4,SI5,RSI,DVI,II1,II2,SR,EVI,RVI,SAVI]);

In [16]:
S2COL=s2_sr_cld_col_eval.map(add_cld_shdw_mask).map(add_bands)

In [17]:
S2COL.first().getInfo()['bands']

[{'id': 'B1',
  'data_type': {'type': 'PixelType',
   'precision': 'double',
   'min': 0,
   'max': 6.5535000000000005},
  'dimensions': [1830, 1830],
  'crs': 'EPSG:32638',
  'crs_transform': [60, 0, 499980, 0, -60, 4100040]},
 {'id': 'B2',
  'data_type': {'type': 'PixelType',
   'precision': 'double',
   'min': 0,
   'max': 6.5535000000000005},
  'dimensions': [10980, 10980],
  'crs': 'EPSG:32638',
  'crs_transform': [10, 0, 499980, 0, -10, 4100040]},
 {'id': 'B3',
  'data_type': {'type': 'PixelType',
   'precision': 'double',
   'min': 0,
   'max': 6.5535000000000005},
  'dimensions': [10980, 10980],
  'crs': 'EPSG:32638',
  'crs_transform': [10, 0, 499980, 0, -10, 4100040]},
 {'id': 'B4',
  'data_type': {'type': 'PixelType',
   'precision': 'double',
   'min': 0,
   'max': 6.5535000000000005},
  'dimensions': [10980, 10980],
  'crs': 'EPSG:32638',
  'crs_transform': [10, 0, 499980, 0, -10, 4100040]},
 {'id': 'B5',
  'data_type': {'type': 'PixelType',
   'precision': 'double',
   'm

In [18]:
# A function to create new Image collections
def collfunc(collection):
  import numpy as np
  list_dates_draft=collection.aggregate_array('system:index').getInfo()
  list_dates=list(np.unique(np.array([x[:8] for x in (list_dates_draft)])))
  col_dict=dict.fromkeys(list_dates)
  for x in list_dates:
    filtered_col=(collection).filter(ee.Filter.stringContains('system:index',x))
    col_dict[x]=filtered_col.set({'date':x})
  return col_dict

In [19]:
coll_dict=collfunc(S2COL)

In [20]:
def valfunc(dictionary,points,bands):
  import pandas as pd
  list_DFs=[]
  for t,col in dictionary.items():
    ls=col.toList(col.size().getInfo())
    my_dict={}
    for b in bands:
      my_dict.update({b:[]})
    my_dict.update({'Lon':[],'Lat':[],'time':[]})
    for point in points:
      coords=ee.Geometry.Point(point,'EPSG:4326')
      my_dict['Lon'].append(point[0])
      my_dict['Lat'].append(point[1])
      my_dict['time'].append(t)
      for B in bands:
        val_list=[]
        for i in range(ls.size().getInfo()):
          val_list.append(ee.Image(ls.get(i)).reduceRegion(reducer=ee.Reducer.mean(),geometry=coords).get(B).getInfo())
        my_dict[B].append(float(pd.DataFrame(val_list).mean().loc[0]))
    list_DFs.append(pd.DataFrame(my_dict))
  last_DF=pd.concat(list_DFs,ignore_index=True)
  return last_DF

In [ ]:
band=['NDVI']
test_df=valfunc(dictionary=coll_dict,points=points,bands=['NDVI'])

In [27]:
test_df

,NDVI,Lon,Lat,time
0,0.070978,46.179167,37.370833,20200920
1,0.083231,46.187500,37.370833,20200920
2,0.238612,46.195833,37.370833,20200920
3,0.125515,46.179167,37.362500,20200920
4,0.181873,46.187500,37.362500,20200920
...,...,...,...,...
155,0.343343,46.137500,37.345833,20201025
156,0.119044,46.145833,37.345833,20201025
157,0.229004,46.095833,37.337500,20201025
158,0.442294,46.104167,37.337500,20201025


In [ ]:
import pandas as pd
coll_dict=collfunc(S2COL)
ls=coll_dict['20200920'].toList(4)
new_ls=[]
points_test=points[80:82]
for i in range(ls.size().getInfo()):
  img=ee.Image(ls.get(i))
  array=ValFunc(image=img,Points=points_test)
  new_ls.append(array)
new_DF=pd.concat(new_ls)
new_ls2=[]
for i in range(len(points)):
  new_ls2.append(new_DF.loc[i].mean())
pd.concat(new_ls2,axis=1).T

KeyboardInterrupt: 

In [ ]:
new_DF=pd.concat([pd.DataFrame(new_ls[0],columns=img.bandNames().getInfo()),pd.DataFrame(new_ls[1],columns=img.bandNames().getInfo()),pd.DataFrame(new_ls[2],columns=img.bandNames().getInfo())])
point0=(new_DF.loc[0].mean())
point1=(new_DF.loc[1].mean())
points=[point0,point1]
pd.concat(points,axis=1).T

,B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B11,B12
0,0.036833,0.040533,0.070867,0.061067,0.1154,0.292333,0.354400,0.353833,0.377933,0.371867,0.193133,0.103867
1,0.045233,0.050400,0.086067,0.077800,0.1387,0.299167,0.343267,0.347400,0.366767,0.345067,0.206967,0.119500


In [ ]:
import numpy as np
np.mean(new_ls[0],new_ls[1])

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
def newfunc(dict_param,points):
  for x,v in dict_param.items():
    ls=v.toList()
    new_ls=[]
    for i in range(ls.size.getInfo()):
      array=ValFunc(ee.Image(ls.get(i)),points)
      new_ls.append(array)
    means=(new_ls[0]+new_ls[1]+new_ls[2]+new_ls[3])/4
    means_DF=pd.DataFrame(means,columns=img.bandNames().getInfo())
    means_DF['Lon']=pd.Series(x[0] for x in points)
    means_DF['Lat']=pd.Series(x[1] for x in points)


In [ ]:
mydict={'Lon':[],'Lat':[],'B4':[]}
for x in points:
  coordinate=ee.Geometry.Point(coords=x,proj='EPSG:4326')
  # B2=float(geemap.ee_to_numpy(S2.first().select('B4'),region=coordinate).reshape(1)[0])
  B4=img.first().reduceRegion(reducer=ee.Reducer.mean(),geometry=coordinate).get('B4')
  mydict['Lon'].append(x[0])
  mydict['Lat'].append(x[1])
  mydict['B4'].append(B4.getInfo())